**This notebook is the executable version of lab note 3.
It answers the following questions:**

Finally, we answer 4 items:

    1. Are successful creators more connected to high-outdegree users than do unsuccessful creators?
    2. Are mavens more connected to successful creators than to unsuccessful creators?
    3. Do successful creators send more non-follow actions towards mavens than to zombies?
    4. Do successful creators send more non-follow actions towards mavens than to stars?

    

In [2]:
#Run parameters
#used to control every run. Can be user to perfom sensitivity checks
path_dir = r"/Users/../Volumes/Raw/"

low_success = 0.5 #below the median: unsuccessful
high_success = 0.9 #top 10% creators with more followers are deemed successful

low_user_outdegree = 0.25 
high_user_outdegree = 0.75
low_user_activity = 0.25 
high_user_activity = 0.75 

activity_filter = 0
days_delta = 7

In [24]:
table_data = pd.read_csv('user_types_ids.csv')

In [3]:
import sys  
import pickle
sys.path.insert(0, '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/')
import numpy as np
import src.utils
from collections import Counter
from src.utils import import_dta, import_tracks_dta,\
gen_active_relations, get_fan_interactions_per_week, calculate_avg_monthly_valence,\
gen_active_relations_prob, get_fan_interactions_per_week_prob, stripplot_prob,\
reaction_probability, follower_list, filter_quantile, sample_creators_music,\
gen_outbound_creators
import numpy as np
import datetime
import pandas as pd
from tqdm import tqdm
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy
import os
from statsmodels.stats.proportion import proportions_ztest

In [4]:
def process_date(date):
    '''convert date format like '2013-w09' to '2013-03-04', i.e. the first day of that week'''
    year = date[0:4]
    week = date[6:]
    day = "1"
    date = "{}-{}-1".format(year, week)
    dt = datetime.datetime.strptime(date, "%Y-%W-%w")
    return dt

# Data Imports

We start by importing the raw data.  `follows_sent`, `comments_sent`, `shares_sent`, `likes_sent` and `messages_sent` contains data pn the promotional activities that the 35k users tracked in the dataset directed to other users. It includes the `user_id`, the `fan_id` and the `date_sent` which identifies the date when the prom. activity was sent. `users_info_1st` shows the type of user (creator or non-creator, which is identified by a blank) and the date the user entered the platform, for every user that sent or received prom. activities from any of the 35k users tracked in this dataset, while `users_info` contains the same information, but pertaining to the 35k users themselves.

`follows_received` contains information on the follows received by the 35k users and will be used to generate the successful/unsuccessful groups of content creators.

In [5]:
#affiliations :follows
#favoritings :likes

#used in filtering:
path_dir = r"/Users/../Volumes/Raw/"
tracks = import_tracks_dta(path_dir, "12sample_tracks.dta");

#these are the actions sent to 
follows_sent = import_dta(path_dir, "12sample_affiliations_sent.dta");
comments_sent = import_dta(path_dir, "12sample_comments_made.dta");
shares_sent = import_dta(path_dir, "12sample_reposts_made.dta");
likes_sent = import_dta(path_dir, "12sample_favoritings_made.dta");
messages_sent = import_dta(path_dir, "12sample_messages_sent.dta");

#Used to track information on the 1st degree connections
user_info_1st = import_dta(path_dir, "12sample_1st_deg_user_infos.dta");
user_info_1st.columns = ['user_id', 'type', 'entered_platform'];
user_info = import_dta(path_dir, "12sample_user_infos.dta");

#Used to compute creator's success measure
follows_received = import_dta(path_dir, "12sample_affiliations_received.dta");

%%%%%%%%%% 12sample_tracks.dta %%%%%%%%%%
(56262, 7)
%%%%%%%%%% 12sample_affiliations_sent.dta %%%%%%%%%%
(800913, 3)
%%%%%%%%%% 12sample_comments_made.dta %%%%%%%%%%
(29258, 4)
%%%%%%%%%% 12sample_reposts_made.dta %%%%%%%%%%
(179329, 4)
%%%%%%%%%% 12sample_favoritings_made.dta %%%%%%%%%%
(527701, 4)
%%%%%%%%%% 12sample_messages_sent.dta %%%%%%%%%%
(11091, 3)
%%%%%%%%%% 12sample_1st_deg_user_infos.dta %%%%%%%%%%
(670746, 3)
%%%%%%%%%% 12sample_user_infos.dta %%%%%%%%%%
(35000, 3)
%%%%%%%%%% 12sample_affiliations_received.dta %%%%%%%%%%
(432503, 3)


Indegree and outdegree information.

The function below import the outdegree dataset. Because the raw version of those dataset are too large to be processed in memory, we preprocessed them in a separate script.

In [5]:
# Aggregates preprocessed outdegree of 1st degree users
def import_outdegree(path='/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/'):
    d = {}
    for i in range(6):
       d[str(i)] = pd.read_pickle(os.path.join(path,'{}.pkl'.format(i))) 
       d[str(i)]['created_at'] =  pd.to_datetime(d[str(i)]['created_at'])
       d[str(i)]['created_at'] = pd.to_datetime(d[str(i)]['created_at']).dt.floor('d')
       d[str(i)] = d[str(i)].groupby(['sender_id', 'created_at'], as_index = False).size() 
    
    data_outdegree = pd.concat([d['0'], d['1'], d['2'], d['3'], d['4'], d['5']])
    #data_outdegree.set_index('created_at', inplace = True)
    return data_outdegree

In [6]:
data_outdegree = import_outdegree()

In [7]:
#data_outdegree = data_outdegree.groupby(['sender_id','created_at'], as_index = False).size()

# Preprocessing

## Creator ids, successful and unsucessful creators

Next, we define three lists of ids: one with the ids from the content creators, according to the `users_info` table, one with the ids of successful creators and the last one with the ids of the unsuccessful ones.

Let's start with a list of the id of creators. We also create a dataset with containing information on creators only.

In [8]:
mask = (tracks.track_available == 1) & (tracks.public == 't')
creator_ids = tracks[mask].user_id.unique()

creators = tracks[(tracks.track_available == 1) & (tracks.public == 't')]

#mask = user_info.type == 'creator'
#creator_ids = user_info[mask].user_id.unique()

#creators = user_info[user_info.type == 'creator']

## Putting together a dataset with the promotional activities made by content creators.

The function `gen_actions_sent_df` creates a dataframe with all the promotional activities that content creators sent to users.

In [9]:
def gen_actions_sent_df(follows_sent, shares_sent, likes_sent, comments_sent, messages_sent, creator_ids = creator_ids):
    '''
    Creates dataframe containing the actions that content creators send to users.
        Attributes:
                    follows_sent:  dataframe with the follows sent by the 35k users.
                    shares_sent:   dataframe with the shares sent by the 35k users.
                    likes_sent:    dataframe with the likes sent by the 35k users.
                    comments_sent: dataframe with the comments sent by the 35k users.
                    messages_sent: dataframe with the messages sent by the 35k users.
                    creator_ids:   list with content creator ids. If not none, is used to
                                   filter out activities from non creators.
    '''
    
    follows_sent['outbound_activity'] = 'follow'
    follows_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    if 'song_id' in shares_sent.columns:
        shares_sent.drop(columns=["song_id"])
    shares_sent = shares_sent[['reposter_id', "owner_id", 'created_at']]
    shares_sent['outbound_activity'] = 'share'
    shares_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    if 'track_id' in likes_sent.columns:
        likes_sent.drop(columns=["track_id"], inplace=True)
    likes_sent['outbound_activity'] = 'like'
    likes_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    if 'track_id' in comments_sent.columns:
        comments_sent.drop(columns=["track_id"], inplace=True)
    comments_sent['outbound_activity'] = 'comment'
    comments_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    messages_sent["outbound_activity"] = 'message'
    messages_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']
    df = pd.concat([follows_sent, shares_sent, likes_sent, comments_sent, messages_sent])


    if type(creator_ids) == numpy.ndarray:
        df = df[df['user_id'].isin(creator_ids)]
        
    df['week_yr'] = df.date_sent.dt.strftime('%Y-w%U')
    df = df.loc[df['user_id'] != df['fan_id'],:]

    return df

In [10]:
actions_sent = gen_actions_sent_df(follows_sent, shares_sent, likes_sent, comments_sent,
                                     messages_sent, creator_ids = None)
actions_sent = actions_sent.loc[actions_sent.user_id.isin(creators.user_id.unique())]

In [11]:
active_users_ids = actions_sent.groupby('user_id', as_index = False).size()
mask = active_users_ids['size']>= activity_filter
active_users_ids = active_users_ids[mask].user_id.unique()

In [12]:
def successful_creators_followers(follows_received, base_date = datetime.datetime(2016, 5, 30, 0, 0), perc1 = None, perc2 = None, subset_creators = None):
    '''Classifies content creators in successful or unsuccessfull
        Arguments:
                    follows_received: dataframe containing the follows received by content creators
                    base date:        date, in datetime.datetime(YYYY, M, DD, H, M) format, in which the number 
                                      of followers per creator is calculated.
                    perc1:            the threshold used to classify unsuccessful content creators. Creator having 
                                      total followers below the number dictated by this threshold, at the base date,
                                      are classified as unsuccessful 
                    perc2:            the threshold used to classify successful content creators. Creator having 
                                      total followers above the number dictated by this threshold, at the base date,
                                      are classified as successful
                    subset_creators:  a pd.DataFrame containing the creators. If is it available, it will be used to 
                                      filter out non creators and to make sure creators with 0 followers are part of
                                      the resulting dataset.
        
    '''
    print(base_date)

    if 'inbound_activity' not in follows_received.columns:
        follows_received.columns = ['fan_id', 'user_id', 'date_sent']

    mask = (follows_received['date_sent'] < base_date)

    df = follows_received[mask].groupby('user_id', as_index=False).agg({'fan_id': pd.Series.nunique})
    df.columns = ['user_id', 'followers']

    
    if type(subset_creators) == pd.DataFrame:
        print('subsetting...')
        df.set_index('user_id', inplace = True)
        df = df.reindex(subset_creators.user_id.unique())
        df.fillna(0, inplace = True)
        df.reset_index(inplace = True)
        df.columns = ['user_id', 'followers']
        
    mask = df.user_id.isin(active_users_ids)
    df = df[mask]

    low = np.quantile(df.followers, perc1)
    high = np.quantile(df.followers, perc2)

    print("High influencer boundary: {}".format(high))
    print("Low influencer boundary: {}".format(low))

    mask = (df["followers"] <= low) | (df["followers"] >= high)
    
    unsuccessful_creator_ids = df.loc[df["followers"] <= low].user_id.unique()
    successful_creator_ids = df.loc[df["followers"] >= high].user_id.unique()

    return unsuccessful_creator_ids, successful_creator_ids


In [13]:
unsuccessful_ids, successful_ids = successful_creators_followers(follows_received, 
                                                        perc1 = low_success, perc2 = high_success, subset_creators = creators)


2016-05-30 00:00:00
subsetting...
High influencer boundary: 81.0
Low influencer boundary: 13.0


In [14]:
print(len(unsuccessful_ids))
print(len(successful_ids))

1959
389


In [15]:
creators.user_id.nunique()

4604

## Filter only actions that were sent to non-fans

We merge the `actions_sent` dataset with a table containing the date each fan started following the creator.

In [16]:
follows_received.columns = ['fan_id', 'user_id', 'date_sent']
followers = follows_received[["fan_id", "user_id", "date_sent"]]
followers.columns = ["fan_id", "user_id", "follower_since"]

actions_sent = actions_sent.merge(followers, right_on = ['user_id', 'fan_id'],
                                      left_on = ['user_id', 'fan_id'], how = 'left')

Since we are interested in acquisition campaings, we need to produce a dataset that exclude actions targetting fans.
We do that using filters based on the date of the action and the date that the user became a fan of the content creator. The resulting dataframe is named `actions_sent_non_fans`.

We then filter only actions that happened before the user follows the content creator.

In [17]:
mask = (actions_sent.date_sent < actions_sent.follower_since) | (actions_sent.follower_since.isnull())
actions_sent_non_fans =  actions_sent[mask]
actions_sent_non_fans['week_yr_date'] = actions_sent_non_fans.week_yr.apply(lambda x: process_date(x))

/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_67304/3855957735.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_sent_non_fans['week_yr_date'] = actions_sent_non_fans.week_yr.apply(lambda x: process_date(x))


## Outdegree level

Originally, we only have outdegree information on users that follow at least one user. The function below inputs an outdegree of 0 to users that are following anyone.

In [18]:
#CHANGE DOCUMENTATION
def outdegree_level(data, date, user_info = user_info_1st):
    
    '''
    This function returns the membership table at date equals `date`. Every user that interacted with the 35k tracked 
    users and entered the platform before `date` is present in the table, even if it has indegree 0.
    arguments:
              data:           the indegree dataset.
              user_info:      the dataset containing all the users that interacted with the 35k users tracked.
    '''
    
    data = data[data.created_at.dt.floor('d')<=date]
    data = data.groupby('sender_id').agg({'size':'sum'}).compute()
    
    #merge with user info to obtain users that are not followed by anyone at the current date
    data = user_info_1st.merge(data, left_on = 'user_id', right_on = 'sender_id', how= 'outer')
    data.loc[data['size'].isnull(), 'size'] = 0
    data = data[['user_id', 'size', 'entered_platform']].set_index('user_id')
    
    #filter out users that didnt exist in the current date
    mask = data['entered_platform'].dt.floor('d') <= date
    data = data.loc[mask]
    
    mask = (data['size']>0)
    data.loc[~mask, 'size'] = 0 
    
    return data

In [19]:
#dask is a python api with objects optimized for big data (user directed acyclic graphs). 
dask_outdegree = dd.from_pandas(data_outdegree, npartitions = 3)

In [20]:
last_day =  max(actions_sent.date_sent.dt.floor('d').unique())
user_outdegree = outdegree_level(dask_outdegree, last_day, user_info = user_info_1st)

Now we classify the creator in successful and unsuccessful, according to the threshold defined in the beggining of this notebook.

In [20]:
def user_outdegree_level(oudegree_data, perc1 = None, perc2 = None):
    '''Classifies content creators in successful or unsuccessfull
        Arguments:
                    oudegree_data:    dataframe containing the fans followers at 
                    perc1:            the threshold used to classify unsuccessful content creators. Creator having 
                                      total followers below the number dictated by this threshold, at the base date,
                                      are classified as unsuccessful 
                    perc2:            the threshold used to classify successful content creators. Creator having 
                                      total followers above the number dictated by this threshold, at the base date,
                                      are classified as successful
    '''

    df = oudegree_data.reset_index().iloc[:,:2]
    df.columns = ['user_id', 'followers']

    low = np.quantile(df.followers, perc1)
    high = np.quantile(df.followers, perc2)

    print("High outdegree boundary: {}".format(high))
    print("Low outdegree boundary: {}".format(low))
    
    low_outdegree_ids = df.loc[df["followers"] <= low].user_id.unique()
    high_outdegree_ids = df.loc[df["followers"] >= high].user_id.unique()
    
    
    df['outdegree_level'] = df.user_id.apply(
        lambda x: 'high' if x in high_outdegree_ids else ('low' if x in low_outdegree_ids else None))

    
    return df, low_outdegree_ids, high_outdegree_ids

In [21]:
user_outdegree_df, low_outdegree_ids, high_outdegree_ids = user_outdegree_level(user_outdegree,
                perc1 = low_user_outdegree, perc2 = high_user_outdegree)

High outdegree boundary: 162.0
Low outdegree boundary: 17.0


In the cell below, we create a list with unique ids from users that appear in the oudegree level table. This will later be uses to construct a flow-chart indicating how we lose data based on filters and operations.

In [22]:
receiver_ids = user_outdegree.index.unique()

## Non-follow Actions level

The activity level is defined as the number of actions performed by users. It is important to notice that we only observe actions targeting the 35k users that joined in march 2012. We consider this measure a proxy for the real activity level.

Let's begin by creating a dataset with all action received by those 35k users.

In [21]:
comments_received_35k = import_dta(path_dir, "12sample_comments_received.dta");
shares_received_35k = import_dta(path_dir, "12sample_reposts_received.dta");
likes_received_35k = import_dta(path_dir, "12sample_favoritings_received.dta");
messages_received_35k = import_dta(path_dir, "12sample_messages_received.dta");

if 'song_id' in shares_received_35k:
        shares_received_35k.drop(columns=["song_id"])
shares_received_35k = shares_received_35k[['reposter_id', "owner_id", 'created_at']]
shares_received_35k['outbound_activity'] = 'share'
shares_received_35k.columns = ['fan_id', 'user_id', 'date_sent', 'outbound_activity']

if 'track_id' in likes_received_35k:
        likes_received_35k = likes_received_35k.drop(columns=["track_id"])
likes_received_35k['outbound_activity'] = 'like'
likes_received_35k.columns = ['fan_id', 'user_id', 'date_sent', 'outbound_activity']

if 'track_id' in comments_received_35k:
        comments_received_35k = comments_received_35k.drop(columns=["track_id"])
comments_received_35k['outbound_activity'] = 'comment'
comments_received_35k.columns = ['fan_id', 'user_id', 'date_sent', 'outbound_activity']

messages_received_35k["outbound_activity"] = 'message'
messages_received_35k.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

user_activity_data_35k = pd.concat([shares_received_35k, likes_received_35k, comments_received_35k, messages_received_35k])

%%%%%%%%%% 12sample_comments_received.dta %%%%%%%%%%
(21386, 4)
%%%%%%%%%% 12sample_reposts_received.dta %%%%%%%%%%
(83013, 4)
%%%%%%%%%% 12sample_favoritings_received.dta %%%%%%%%%%
(286903, 4)
%%%%%%%%%% 12sample_messages_received.dta %%%%%%%%%%
(17364, 3)


In [24]:
path_dir_2 = r'/Users/../Volumes/Alter_outbound_activities/'

comments_received_c = import_dta(path_dir_2, "12sample_1st_deg_comments_made.dta");
shares_received_c = import_dta(path_dir_2, "12sample_1st_deg_reposts_made.dta");
likes_received_c = import_dta(path_dir_2, "12sample_1st_deg_favoritings_made.dta");
messages_received_c = import_dta(path_dir_2, "12sample_1st_deg_messages_sent.dta");

%%%%%%%%%% 12sample_1st_deg_comments_made.dta %%%%%%%%%%
(21463011, 4)
%%%%%%%%%% 12sample_1st_deg_reposts_made.dta %%%%%%%%%%
(18953640, 4)
%%%%%%%%%% 12sample_1st_deg_favoritings_made.dta %%%%%%%%%%
(86793370, 4)
%%%%%%%%%% 12sample_1st_deg_messages_sent.dta %%%%%%%%%%
(16824074, 3)


In [25]:
if 'song_id' in shares_received_c:
        shares_received_c.drop(columns=["song_id"])
shares_received_c = shares_received_c[['reposter_id', "owner_id", 'created_at']]
shares_received_c['inbound_activity'] = 'share'
shares_received_c.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

if 'track_id' in likes_received_c:
        likes_received_c = likes_received_c.drop(columns=["track_id"])
likes_received_c['inbound_activity'] = 'like'
likes_received_c.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

if 'track_id' in comments_received_c:
        comments_received_c = comments_received_c.drop(columns=["track_id"])
comments_received_c['inbound_activity'] = 'comment'
comments_received_c.columns_c = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

messages_received_c["outbound_activity"] = 'message'
messages_received_c.columns = ['user_id', 'fan_id', 'date_sent', 'inbound_activity']

/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_64965/1613121866.py:15: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  comments_received_c.columns_c = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']


In [26]:
user_activity_data_c = pd.concat([shares_received_c, likes_received_c, comments_received_c, messages_received_c])

In [27]:
def user_activity_level(user_activity_data, receiver_ids, perc1 = None, perc2 = None):
    '''Classifies users based on the amount of non-follow activities
        Arguments:
                    user_activity_data:   dataframe containing the user activities 
                    base date:             date, in datetime.datetime(YYYY, M, DD, H, M) format, in which the number 
                                           of followers per creator is calculated.
                    perc1:                 the threshold used to classify unsuccessful content creators. Creator having 
                                           total followers below the number dictated by this threshold, at the base date,
                                           are classified as unsuccessful 
                    perc2:                 the threshold used to classify successful content creators. Creator having 
                                           total followers above the number dictated by this threshold, at the base date,
                                           are classified as successful
    '''

    df = user_activity_data.groupby('fan_id', as_index = True).size()
    df = df.reindex(receiver_ids)
    print(df.shape)
    df = df.reset_index()
    df.columns = ['user_id', 'activities_performed']
    
    df.loc[df.activities_performed.isna(), 'activities_performed'] = 0
    #classification should happen after this

    low = np.quantile(df.activities_performed, perc1)
    high = np.quantile(df.activities_performed, perc2)

    print("High activity boundary: {}".format(high))
    print("Low activity boundary: {}".format(low))
    
    low_activity_ids = df.loc[df["activities_performed"] <= low].user_id.unique()
    high_activity_ids = df.loc[df["activities_performed"] > high].user_id.unique()
    
    df['activity_level'] = df.user_id.apply(
    lambda x: 'high' if x in high_activity_ids else ('low' if x in low_activity_ids else None))

    return df, low_activity_ids, high_activity_ids

In [28]:
activity_level, low_activity_ids, high_activity_ids = user_activity_level(user_activity_data_c, receiver_ids, 
                perc1 = low_user_activity, perc2 = high_user_activity)

(670746,)
High activity boundary: 70.0
Low activity boundary: 1.0


Once more we create an object containing the unique ids of users in the resulting dataset. This will be used in a flow-chart, as explained.

In [29]:
activity_ids = np.append(low_activity_ids, high_activity_ids)

# Priori response probabilities

In [26]:
# table_data = user_outdegree_df.merge(activity_level, left_on = 'user_id', right_on = 'user_id', how = 'inner')
# table_data.groupby(['outdegree_level', 'activity_level']).size()
# table_data['Type'] = table_data.apply(lambda x: 
#     'f_a' if (x.outdegree_level == 'low') & (x.activity_level == 'high')
#      else ('Hermit' if (x.outdegree_level == 'low') & (x.activity_level == 'low')
#      else ('Observer' if (x.outdegree_level == 'high') & (x.activity_level == 'low')
#      else ('w_a' if (x.outdegree_level == 'high') & (x.activity_level == 'high')
#      else 'other'))), axis=1)

hermit_ids = table_data.loc[table_data.Type =='Hermit'].user_id.unique()
w_a_ids = table_data.loc[table_data.Type =='w_a'].user_id.unique()
f_a_ids = table_data.loc[table_data.Type =='f_a'].user_id.unique()
observer_ids = table_data.loc[table_data.Type =='Observer'].user_id.unique()

In [27]:
actions_sent['user_type'] = actions_sent.fan_id.apply(lambda x: 'f_a' if x in f_a_ids else 
                          ('Hermit' if x in hermit_ids else
                          ('Observer' if x in observer_ids else
                          ('w_a' if x in w_a_ids else 'other'))))

## classify content creators
actions_sent['creator_type'] = actions_sent.user_id.apply(
                               lambda x: 'successful' if x in successful_ids else 
                               ('unsuccessful' if x in unsuccessful_ids else 'other'))

In [28]:
import datetime
# Target Creation
delta = datetime.timedelta(days = days_delta)
mask = (actions_sent_non_fans['follower_since'] <= (actions_sent_non_fans['date_sent'] + delta))

response_df = actions_sent_non_fans.copy()
response_df.loc[mask, 'reward'] = 1
mask = response_df['reward'].isnull()
response_df.loc[mask, 'reward'] = 0

In [30]:
attempts = response_df.groupby(['creator_type', 'user_type'], as_index = False).size()
rewards = response_df.groupby(['creator_type', 'user_type'], as_index = False).agg({'reward' : 'sum'})

priori_prob_df = rewards.merge(attempts)
priori_prob_df['P_resp_prob'] = priori_prob_df['reward']/priori_prob_df['size']
priori_prob_df 

KeyError: 'creator_type'

# Priori repost probabilities

In [36]:
user_activity_data_35k

,fan_id,user_id,date_sent,outbound_activity
0,33635491,38135744,2013-03-05 05:50:34,share
1,38321910,38123852,2013-03-05 23:11:40,share
2,38263941,38263941,2013-03-06 17:43:46,share
3,38470533,38470533,2013-03-09 13:10:51,share
4,6746343,38145276,2013-03-11 06:32:23,share
...,...,...,...,...
17359,38454344,45734072,2015-06-25 12:58:18,message
17360,38247575,160132541,2015-06-27 18:11:42,message
17361,37923935,160510643,2015-06-30 05:25:18,message
17362,37875920,160559068,2015-06-30 10:46:04,message


In [22]:
user_activity_data_35k['day_yr_date'] = user_activity_data_35k.date_sent.dt.normalize()

reposts_df = user_activity_data_35k.loc[user_activity_data_35k.outbound_activity == 'share']

In [31]:
actions_sent['user_type'] = actions_sent.fan_id.apply(lambda x: 'f_a' if x in f_a_ids else 
                          ('Hermit' if x in hermit_ids else
                          ('Observer' if x in observer_ids else
                          ('w_a' if x in w_a_ids else 'other'))))

## classify content creators
actions_sent['creator_type'] = actions_sent.user_id.apply(
                                            lambda x: 'successful' if x in successful_ids else 
                                            ('unsuccessful' if x in unsuccessful_ids else 'other'))

In [32]:
mask = (actions_sent.date_sent >= actions_sent.follower_since)
actions_sent_to_fans = actions_sent[mask]
actions_sent_to_fans['week_yr_date'] = actions_sent_to_fans.week_yr.apply(lambda x: process_date(x))

repost_prob_df = actions_sent_to_fans # filter for follower only
repost_prob_df.sort_values(['user_id', 'fan_id'], inplace = True)
repost_prob_df['reward_repost'] = np.nan
for user_id in tqdm(repost_prob_df.user_id.unique()):
    for fan_id in repost_prob_df.loc[repost_prob_df.user_id == user_id].fan_id.unique():
        repost_prob_df.loc[(repost_prob_df.user_id == user_id)&
                              (repost_prob_df.fan_id == fan_id),'reward_repost'] =\
        repost_prob_df.loc[(repost_prob_df.user_id == user_id)&
                              (repost_prob_df.fan_id == fan_id)].date_sent.apply(
        lambda x : 1 if 
    (reposts_df.loc[
    (reposts_df.user_id == user_id)&
    (reposts_df.fan_id == fan_id)&
    (reposts_df.day_yr_date > x)&   
    (reposts_df.day_yr_date <= x + datetime.timedelta(days=days_delta))]).shape[0]>0 else 0)
                                          
repost_prob_df.sort_values(by='date_sent', inplace = True)

/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_67304/699805651.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_sent_to_fans['week_yr_date'] = actions_sent_to_fans.week_yr.apply(lambda x: process_date(x))
/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_67304/699805651.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repost_prob_df.sort_values(['user_id', 'fan_id'], inplace = True)
/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_67304/699805651.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [33]:
user_activity_data_35k.outbound_activity.value_counts()

like       286903
share       83013
comment     21386
message     17364
Name: outbound_activity, dtype: int64

In [34]:
attempts = repost_prob_df.groupby(['creator_type', 'user_type'], as_index = False).size()
rewards = repost_prob_df.groupby(['creator_type', 'user_type'], as_index = False).agg({'reward_repost' : 'sum'})

priori_rep_df = rewards.merge(attempts)
priori_rep_df['P_rep_prob'] = priori_rep_df['reward_repost']/priori_rep_df['size']
priori_rep_df 

,creator_type,user_type,reward_repost,size,P_rep_prob
0,other,Hermit,1.0,624,0.001603
1,other,Observer,0.0,173,0.000000
2,other,f_a,0.0,73,0.000000
3,other,other,77.0,12259,0.006281
4,other,w_a,17.0,2728,0.006232
5,successful,Hermit,0.0,335,0.000000
6,successful,Observer,0.0,252,0.000000
7,successful,f_a,13.0,194,0.067010
8,successful,other,100.0,13856,0.007217
9,successful,w_a,372.0,11533,0.032255


In [35]:
sum(priori_rep_df['size'])

44769

In [41]:
priori_rep_df.loc[priori_rep_df.creator_type == 'successful']

,creator_type,user_type,reward_repost,size,P_rep_prob
5,successful,Hermit,0.0,335,0.000000
6,successful,Observer,0.0,252,0.000000
7,successful,f_a,13.0,194,0.067010
8,successful,other,100.0,13856,0.007217
9,successful,w_a,372.0,11533,0.032255


In [ ]:
repost_avg_df = actions_sent.copy()
repost_avg_df.sort_values(['user_id', 'fan_id'], inplace = True)
repost_avg_df['reward_repost'] = np.nan
for user_id in tqdm(repost_avg_df.user_id.unique()):
    for fan_id in repost_avg_df.loc[repost_avg_df.user_id == user_id].fan_id.unique():
        repost_avg_df.loc[(repost_avg_df.user_id == user_id)&
                              (repost_avg_df.fan_id == fan_id),'reward_repost'] =\
        repost_avg_df.loc[(repost_avg_df.user_id == user_id)&
                              (repost_avg_df.fan_id == fan_id)].date_sent.apply(
        lambda x : 
    (reposts_df.loc[
    (reposts_df.user_id == user_id)&
    (reposts_df.fan_id == fan_id)&
    (reposts_df.day_yr_date > x)&   
    (reposts_df.day_yr_date <= x + datetime.timedelta(days=days_delta))]).shape[0])
                                          
repost_avg_df.sort_values(by='date_sent', inplace = True)

# Expected (Indirect) Returns Associated with a Song-Repost

In [ ]:
import pickle as pkl
share_follower_lists = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/alter_follower_lists.pkl')
action_follower_lists = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/alter_follower_lists_2.pkl')

In [ ]:
follower_lists = share_follower_lists
expec_followers = user_activity_data_35k.loc[user_activity_data_35k.outbound_activity == 'share']
expec_followers = expec_followers.loc[expec_followers.user_id.isin(creators.user_id)]

expec_followers = expec_followers.loc[expec_followers.fan_id.isin(follower_lists.index.unique())]
expec_followers['expec_followers'] = np.nan
expec_followers.sort_values(['user_id', 'fan_id'], inplace = True)

follows_received.date_sent = follows_received.date_sent.dt.normalize()

for user_id in tqdm(expec_followers.user_id.unique()):
    follows_received_j = follows_received.loc[follows_received.user_id == user_id]
    for fan_id in expec_followers.loc[expec_followers.user_id == user_id].fan_id.unique():
        fan_follows = follower_lists.loc[follower_lists.index == fan_id].values[0]
        expec_followers.loc[(expec_followers.user_id == user_id)&
                                  (expec_followers.fan_id == fan_id),'expec_followers'] =\
        expec_followers.loc[(expec_followers.user_id == user_id)&
                                  (expec_followers.fan_id == fan_id)].date_sent.apply(
            lambda x : 
            np.sum(np.in1d(
                           follows_received_j.loc[(x <= follows_received_j.date_sent) & 
                                                  (follows_received_j.date_sent<= x + 
                                                   datetime.timedelta(days=days_delta))
                                                 ].values,
                fan_follows)
        ))                                        

In [ ]:
expec_followers['user_type'] = expec_followers.fan_id.apply(lambda x: 'f_a' if x in f_a_ids else 
                          ('Hermit' if x in hermit_ids else
                          ('Observer' if x in observer_ids else
                          ('w_a' if x in w_a_ids else 'other'))))

## classify content creators

expec_followers['creator_type'] = expec_followers.user_id.apply(
                                            lambda x: 'successful' if x in successful_ids else 
                                            ('unsuccessful' if x in unsuccessful_ids else 'other'))
attempts = expec_followers.groupby(['creator_type', 'user_type'], as_index = False).size()
desc = expec_followers.groupby(['creator_type', 'user_type'], as_index = False).agg(expected_followers=('expec_followers', 'mean'),
                               std_expected_followers=('expec_followers', 'std'),
                               max_followers=('expec_followers', 'max'))

expect_return = attempts.merge(desc)

In [ ]:
reposts_df.shape

In [ ]:
priori_act_df

# All activities

In [ ]:
#now using all activities
user_activity_data_35k['day_yr_date'] = user_activity_data_35k.date_sent.dt.normalize()

activities_df = user_activity_data_35k.loc[user_activity_data_35k.outbound_activity.isin(['share', 'comment', 'like'])]

act_prob_df = actions_sent.copy()
act_prob_df.sort_values(['user_id', 'fan_id'], inplace = True)
act_prob_df['reward_repost'] = np.nan
for user_id in tqdm(act_prob_df.user_id.unique()):
    for fan_id in act_prob_df.loc[act_prob_df.user_id == user_id].fan_id.unique():
        act_prob_df.loc[(act_prob_df.user_id == user_id)&
                              (act_prob_df.fan_id == fan_id),'reward_repost'] =\
        act_prob_df.loc[(act_prob_df.user_id == user_id)&
                              (act_prob_df.fan_id == fan_id)].date_sent.apply(
        lambda x : 1 if 
    (activities_df.loc[
    (activities_df.user_id == user_id)&
    (activities_df.fan_id == fan_id)&
    (activities_df.day_yr_date > x)&   
    (activities_df.day_yr_date <= x + datetime.timedelta(days=days_delta))]).shape[0]>0 else 0)
                                          
act_prob_df.sort_values(by='date_sent', inplace = True)

attempts = act_prob_df.groupby(['creator_type', 'user_type'], as_index = False).size()
rewards = act_prob_df.groupby(['creator_type', 'user_type'], as_index = False).agg({'reward_repost' : 'sum'})
 
priori_act_df = rewards.merge(attempts)
priori_act_df['P_rep_prob'] = priori_act_df['reward_repost']/priori_act_df['size']
priori_act_df 

In [ ]:
expec_followers2 = user_activity_data_35k.loc[user_activity_data_35k.outbound_activity.isin(['share', 'comment', 'like'])]
expec_followers2 = expec_followers2.loc[expec_followers2.user_id.isin(creators.user_id)]
follower_list = action_follower_lists

expec_followers2 = expec_followers2.loc[expec_followers2.fan_id.isin(follower_lists.index.unique())]
expec_followers2['expec_followers'] = np.nan
expec_followers2.sort_values(['user_id', 'fan_id'], inplace = True)

for user_id in tqdm(expec_followers2.user_id.unique()):
    follows_received_j = follows_received.loc[follows_received.user_id == user_id]
    for fan_id in expec_followers2.loc[expec_followers2.user_id == user_id].fan_id.unique():
        fan_follows = follower_lists.loc[follower_lists.index == fan_id].values[0]
        expec_followers2.loc[(expec_followers2.user_id == user_id)&
                                  (expec_followers2.fan_id == fan_id),'expec_followers'] =\
        expec_followers2.loc[(expec_followers2.user_id == user_id)&
                                  (expec_followers2.fan_id == fan_id)].date_sent.apply(
            lambda x : 
            np.sum(np.in1d(
                           follows_received_j.loc[(x <= follows_received_j.date_sent) & 
                                                  (follows_received_j.date_sent<= x + 
                                                   datetime.timedelta(days=days_delta))
                                                 ].values,
                fan_follows)
        ))                                        

In [ ]:
expec_followers2['user_type'] = expec_followers2.fan_id.apply(lambda x: 'f_a' if x in f_a_ids else 
                          ('Hermit' if x in hermit_ids else
                          ('Observer' if x in observer_ids else
                          ('w_a' if x in w_a_ids else 'other'))))

## classify content creators

expec_followers2['creator_type'] = expec_followers2.user_id.apply(
                                            lambda x: 'successful' if x in successful_ids else 
                                            ('unsuccessful' if x in unsuccessful_ids else 'other'))
attempts = expec_followers2.groupby(['creator_type', 'user_type'], as_index = False).size()
desc = expec_followers2.groupby(['creator_type', 'user_type'], as_index = False).agg(expected_followers=('expec_followers', 'mean'),
                               std_expected_followers=('expec_followers', 'std'),
                               max_followers=('expec_followers', 'max'))

expect_return2 = attempts.merge(desc)

In [ ]:
expect_return2

# Analysis

In [ ]:
returns_table = priori_prob_df[['creator_type', 'user_type', 'P_resp_prob']].\
merge(priori_rep_df[['creator_type', 'user_type', 'P_rep_prob']]).\
merge(expect_return[['creator_type', 'user_type', 'expected_followers']])

In [ ]:
returns_table['Expected_direct_return'] = returns_table['P_resp_prob']
returns_table['Expected_indirect_return'] = returns_table['P_rep_prob']*returns_table['expected_followers']
returns_table['Total_expected_return'] = returns_table['Expected_direct_return'] + returns_table['Expected_indirect_return']

In [ ]:
returns_table[['creator_type', 'user_type', 'P_resp_prob', 'P_rep_prob',
       'expected_followers',
       'Expected_direct_return', 'Expected_indirect_return',
       'Total_expected_return']].loc[(returns_table.creator_type == 'successful')] #& (returns_table.user_type.isin(['Hermit', 'f_a']))]

In [ ]:
#avg outbound activities from successful creators to the same user
mask = actions_sent.creator_type == 'successful'
size = actions_sent.loc[mask].groupby(['user_id', 'fan_id','user_type'], as_index = False).size()

In [ ]:
size.groupby('user_type')['size'].mean()

In [ ]:
#Response (Follow-Back) Probabilities
df = priori_prob_df

pivoted_df = round(df.pivot(index='creator_type', columns='user_type', values=['P_resp_prob'])*100,3)

# Display the pivoted DataFrame
pivoted_df

In [ ]:
#Response (Follow-Back) Probabilities
df = priori_rep_df

pivoted_df = round(df.pivot(index='creator_type', columns='user_type', values=['P_rep_prob'])*100,2)

# Display the pivoted DataFrame
pivoted_df

In [ ]:
#Response (Follow-Back) Probabilities
df = expect_return

pivoted_df = round(df.pivot(index='creator_type', columns='user_type', values=['expected_followers']),2)

# Display the pivoted DataFrame
pivoted_df

In [ ]:
#Response (Follow-Back) Probabilities
df = expect_return

pivoted_df = round(df.pivot(index='creator_type', columns='user_type', values=['std_expected_followers']),2)

# Display the pivoted DataFrame
pivoted_df

In [ ]:
df = returns_table

pivoted_df = round(df.pivot(index='creator_type', columns='user_type', values=['Total_expected_return']),4)

# Display the pivoted DataFrame
pivoted_df

In [ ]:
df = returns_table

pivoted_df = round(df.pivot(index='creator_type', columns='user_type', values=['Expected_indirect_return']),4)

# Display the pivoted DataFrame
pivoted_df

In [ ]:
make a table wit all rewards: follows, reposts and # of followers. I think I can simply join the tables that I already havefrep

In [ ]:
#table non fans

response_df.merge(reposts_df).merge(expec_followers)

In [ ]:
#table fans

reposts_df.merge(expec_followers)

In [ ]:
repost_avg_df.reward_repost.value_counts(normalize = True)

# Second One Pager

In [ ]:
#Response (Follow-Back) Probabilities
df = priori_act_df

pivoted_df = round(df.pivot(index='creator_type', columns='user_type', values=['P_rep_prob'])*100,2)

# Display the pivoted DataFrame
pivoted_df

In [ ]:
#Response (Follow-Back) Probabilities
df = expect_return2

pivoted_df = round(df.pivot(index='creator_type', columns='user_type', values=['expected_followers']),2)

# Display the pivoted DataFrame
pivoted_df

In [ ]:
#Response (Follow-Back) Probabilities
df = expect_return2

pivoted_df = round(df.pivot(index='creator_type', columns='user_type', values=['std_expected_followers']),2)

# Display the pivoted DataFrame
pivoted_df

In [ ]:
returns_table2 = priori_prob_df[['creator_type', 'user_type', 'P_resp_prob']].\
merge(priori_act_df[['creator_type', 'user_type', 'P_rep_prob']]).\
merge(expect_return2[['creator_type', 'user_type', 'expected_followers']])

returns_table2['Expected_direct_return'] = returns_table2['P_resp_prob']
returns_table2['Expected_indirect_return'] = returns_table2['P_rep_prob']*returns_table2['expected_followers']
returns_table2['Total_expected_return'] = returns_table2['Expected_direct_return'] + returns_table2['Expected_indirect_return']

returns_table2[['creator_type', 'user_type', 'P_resp_prob', 'P_rep_prob',
       'expected_followers',
       'Expected_direct_return', 'Expected_indirect_return',
       'Total_expected_return']].loc[(returns_table2.creator_type == 'successful')] #& (returns_table.user_type.isin(['Hermit', 'f_a']))]

In [ ]:
df = returns_table2

pivoted_df = round(df.pivot(index='creator_type', columns='user_type', values=['Total_expected_return']),4)

# Display the pivoted DataFrame
pivoted_df

In [ ]:
df = returns_table2

pivoted_df = round(df.pivot(index='creator_type', columns='user_type', values=['Expected_indirect_return']),4)

# Display the pivoted DataFrame
pivoted_df

# Multiarmed Bandits

In [ ]:
#create database with direct and indirect rewards

In [ ]:
#MAB dataframe
follower_lists = action_follower_lists
dataframe = actions_sent.copy()
dataframe = dataframe.loc[dataframe.fan_id.isin(follower_lists.index.unique())]

import datetime
# Target Creation
delta = datetime.timedelta(days = days_delta)
mask = (actions_sent['date_sent']<= actions_sent['follower_since']) & (actions_sent['follower_since'] <= (actions_sent['date_sent'] + delta))

dataframe = actions_sent
dataframe.loc[mask, 'reward'] = 1
mask = dataframe['reward'].isnull()
dataframe.loc[mask, 'reward'] = 0

In [ ]:
activities = activities_df.copy()
activities.reset_index(inplace = True)
activities.columns = ['activity_id', 'fan_id', 'user_id', 'date_returned', 'outbound_activity_returned', 'day_yr_date']

In [ ]:
from datetime import timedelta

merged_df = pd.merge(dataframe, activities[['activity_id', 'user_id', 'fan_id', 'date_returned', 'outbound_activity_returned']]
                , left_on=['user_id', 'fan_id'], right_on=['user_id', 'fan_id'], how = 'left')

# Calculate the time difference between dates
merged_df['Date_Diff'] = merged_df['date_returned'] - merged_df['date_sent']

# Drop negative Date_Diff

merged_df = merged_df[(merged_df['Date_Diff'] > timedelta(minutes=0))| merged_df['Date_Diff'].isna()]

# Sort the merged DataFrame by time difference
merged_df.sort_values(by=['user_id', 'fan_id', 'Date_Diff'], inplace=True)

# Keep only the closest match for each unique combination
merged_df.drop_duplicates(subset=['user_id', 'fan_id', 'Date_Diff'], keep='first', inplace=True)

merged_df = merged_df[(merged_df['Date_Diff'] < timedelta(days=7))|(merged_df['Date_Diff'].isna())]

merged_df['activity_reward'] = (merged_df['Date_Diff'].isna() == False)*1.0

merged_df = merged_df.merge(expec_followers2[['user_id', 'fan_id', 'date_sent', 'expec_followers']], 
                left_on = ['user_id', 'fan_id', 'date_returned'], 
                right_on = ['user_id', 'fan_id', 'date_sent'], 
                how = 'left')


In [ ]:
expec_followers2.user_type.value_counts()

In [ ]:
merged_df.loc[(merged_df.date_sent_y.isna() == False)&(merged_df.user_type.isin(['Hermit','f_a']))]

In [ ]:
merged_df.loc[merged_df.expec_followers.isna()== True, 'expec_followers'] = 0

In [ ]:
merged_df = merged_df.loc[merged_df['user_type'].isin(['Hermit', 'f_a'])]
user_len = merged_df.groupby(['user_id']).user_type.apply(set).apply(lambda x: len(x))

merged_df = merged_df.loc[merged_df.user_id.isin(user_len.loc[user_len>1].index)]

In [ ]:
class MultiArmedBandits:
    
    def __init__(self, arm_names, user_id, data):
        self.user_id = user_id
        self.arm_names = arm_names
        self.arms = {k:{'Sent':0,'Reward':0} for k in arm_names}
        self.means = np.array(len(arm_names))
        self.variances = np.array(len(arm_names))
        self.total_trials = 0 
        self.rewards = 0
        self.data = data.loc[data.user_id == user_id]
        self.next_arm_pull = None
        self.N = self.data.shape[0]
        self.direct_return = 0
        self.indirect_return = 0
    
    def choose_arm(self):
        '''sample from a beta based on self.arms'''
        stats = {k:np.random.beta(self.arms[k]['Reward']+1, self.arms[k]['Sent']+1) for k in self.arms}
        self.next_arm_pull = max(stats, key=stats.get)
    
    def sample_reward(self):
        '''filter data based on arm and sample a random row'''
        i = np.random.randint(len(self.data.loc[self.data.user_type == self.next_arm_pull]))
        self.direct_return += self.data.loc[self.data.user_type == self.next_arm_pull].iloc[i]['reward'] 
        self.indirect_return += self.data.loc[self.data.user_type == self.next_arm_pull].iloc[i]['expec_followers']
        return self.data.loc[self.data.user_type == self.next_arm_pull].iloc[i]['reward']
 
    def update_arms(self):
        self.arms[self.next_arm_pull]['Sent'] += 1
        self.arms[self.next_arm_pull]['Reward'] += self.sample_reward()
                
    def estimate_moments(self):
        Sent = np.array([a[1]['Sent'] for a in self.arms.items()])
        Reward = np.array([a[1]['Reward'] for a in self.arms.items()])
        
        alpha = Reward + 1 
        beta = (Sent-Reward)+1

        mean = alpha/(alpha+beta)
        variance = (alpha*beta)/((alpha+beta+1)*((alpha+beta)**2))
        
        self.means = np.transpose(np.around(mean,3))
        self.variances = np.transpose(np.around(np.sqrt(variance),3))

In [ ]:
d = {}

for user_id in tqdm(response_df.user_id.unique()):
    d[user_id] = MultiArmedBandits(['Hermit', 'f_a'], user_id, merged_df)
    for _ in range(d[user_id].N):
        d[user_id].choose_arm()
        d[user_id].update_arms()
    d[user_id].estimate_moments()

In [ ]:
activities_df

In [ ]:
user_activity_data_35k

In [ ]:
expec_followers2

In [ ]:
share_follower_lists